In [ ]:
import os
import pandas as pd 
import requests
from api_key import apikey
import matplotlib.pyplot as plt

In [ ]:
class FredPy:

    def __init__(self, token=None):
        self.token = token
        self.url = f"https://api.stlouisfed.org/fred/series/observations?"+ \
                    "series_id={seriesID}&api_key={key}&file_type=json&" + \
                    "observation_start={start}&observation_end={end}&units={units}"

    def set_token(self, token):
        self.token = token

    def get_series(self, seriesID, start, end, units):
        url_formatted = self.url.format(seriesID=seriesID, key=self.token, start=start, end=end, units=units)
        response = requests.get(url_formatted)
        if (self.token):
            if (response.status_code == 200):
                data = pd.DataFrame(response.json()["observations"])[["date","value"]]\
                                        .assign(date=lambda cols: pd.to_datetime(cols['date']))\
                                        .assign(value= lambda cols: cols['value'].astype(float))\
                                        .rename(columns={"value":seriesID})
                return data
            else:
                raise Exception("Bad response from the API, status_code = {}". format(response.status_code))
        else:
            raise Exception("API token not configured correctly")

In [ ]:
#Initiate Fredpy object
fredpy = FredPy()

#Set the API key
fredpy.set_token(apikey)

gdp_data = fredpy.get_series (seriesID="GDP",
                          start="2020-01-01",
                          end="2025-07-03",
                          units="pc1")

unemp_data = fredpy.get_series (seriesID="UNRATE",
                          start="2020-01-01",
                          end="2025-07-03",
                          units="lin")

gdp_data
unemp_data

In [ ]:
#visualize
plt.figure(figsize=(12,7))
plt.plot(gdp_data["date"], gdp_data["GDP"], label= "GDP")
plt.plot(unemp_data["date"], unemp_data["UNRATE"], label="Unemp_rate")
plt.legend()
plt.title("Data from FRED API")
plt.show()